## A Chatbot using GPT and a Database
This allows multiple chatbot types (e.g. a health coach and a learning assistant) to be created. Multiple chatbot instances can be created per chatbot type (e.g. for User X and User Y). Both, type and instance are stored and referenced with an ID (e.g. with a UUID) in the database.

This can support the deployment of chatbots in a web backend (state-less). For example, the UUIDs of the type and instance can be read as URL parameters from a URL that users have received from you.

A chatbot is created with the following arguments.
- database_file: File of SQLite (in Folder data/)
- type_id: Reference to chatbot type
- instance_id: Reference to chatbot instance (typically one per user - however, may also be shared by multiple users)
- type_role: Role of chatbot type (will be turned into a first prompt with role:system)
- instance_context: Context of chatbot instance (will be turned into a second prompt with role:system)
- instance_starter: Will be used to generate an initial message to the user (will be turned into a final prompt with role:system)

The following functions are meant to be used from an application (e.g. from controllers of a REST API).
- conversation_retrieve(with_system=False): Retrieve the previous conversation history (default: without prompts with role:system)
- start(): Returns an initial message to the user (Resulting from instance_starter prompt)
- respond(user_says): Returns an assistance response to user_says
- info_retrieve(): Returns the chatbot name, type role and instance context
- reset(): Resets the conversation so far

#### OpenAI API Key and Model

Create file chatbot/openai.py with the following content\
    OPENAI_KEY = "your OpenAI API key"\
    OPENAI_MODEL = "gpt-3.5-turbo-16k"\
(You may rename the file chatbot/openai_template.py to openai.py and set the keys there)

In [1]:
from chatbot.chatbot import Chatbot

#### Create a chatbot "Coach" for user X

In [2]:
bot_type = """
Du bist ein achtsamer, respektvoller, freundlicher und wohlwollender Coach. 
Du hilfst Personen, ihre Steuererklärung regelkonform auszufüllen.  
Die Personen haben dank Dir ein Gefühl der Sicherheit, dass sie ihre Steuererklärung korrekt ausgefüllt haben. 
Du hilst ihnen, dass sie alle Dokumente zur Verfügung haben, die sie zum Ausfüllen der Steuererklärung brauchen. 
Du informierst sie darüber, welche Informationen aus den verschiedenen Dokumenten wo in der Steuererklärung eingetragen werden müssen. 
Du informierst sie über mögliche Abzüge, die sie geltend machen können. 
"""
bot_instance = """
Du führst nun ein Gespräch mit einer Person, die in der Schweiz im Kanton Aargau steuerpflichtig ist. 
Diese Person hat ein Einkommen aus unselbstständiger Tätigkeit als zu 100%% Büroangestellte Person.
Diese Person besitzt und bewohnt eine Eigentumswohnung mit einer Hypothek bei der Bank namens Swizzy. 
Sie besitzt Aktien von Nestlé, Roche und Swiss Re. 
Sie hat zwei betreuungspflichtige Kinder im Alter von drei Jahren. 
Ausserdem hat sie ein Fahrrad, mit dem sie ihren Arbeitsweg zurücklegt.
Hilf dieser Person, folgendes in ihrer Steuererklärung anzugeben:
Einkünfte aus unselbstständigem Erwerb, betreuungspflichtige Kinder inklusive der Abzüge, Konto bei der Bank, Aktien inklusive deren Steuerwerte.
Im Rahmen dieser Coaching-Session braucht die Person keine anderen Angaben zu machen.
Hilf dieser Person dabei, die dafür benötigten Informationen aus folgenden Dokumenten korrekt in die Steuererklärung zu übertragen: 
Lohnausweis, Kontoauszug, Zinsauszug, Depotauszug.
Im Rahmen dieser Coaching-Session braucht die Person keine anderen Dokumente.
"""
bot_starter = """
<p>Bei Antworten:</p>
<ol>
    <li>Emojis einbinden, wenn es passt. 😊</li>
    <li>Stelle sicher, dass die Antworten vollständig und sind.</li>
    <li>Verwende <b>&lt;ol&gt;/&lt;ul&gt;</b> mit <b>&lt;li&gt;</b>, um Informationen in Listenform zu präsentieren, wenn es passt.</li>
    <li>Wenn es die Möglichkeit gibt, mehr als eine Information oder mehr als ein Feedback zu geben, teile Deine Antwort in mehrere <b>&lt;p&gt;</b>-Elemente auf, um eine bessere Klarheit zu gewährleisten.</li>
    <li>Formatiere alle Antworten immer mit gültigem HTML: z.B. <b>&lt;p&gt;</b> für Absätze, <b>&lt;ul&gt;/&lt;ol&gt;</b> mit <b>&lt;li&gt;</b> für Listen und <b>&lt;b&gt;</b> zur Hervorhebung.</li>
    <li>Halte einen humorvollen Ton bei. Halte es kurz, aber opfere nicht die Klarheit für Kürze.</li>
    <li>Antworte in korrektem Deutsch.</li>
</ol>
Begrüsse jetzt die Person freundlich und frage, wie Du die Person ansprechen darfst.
Verwende den Namen der Person im geschlechtsneutralem Gespräch in Du-Form.
"""

In [3]:
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id="db4dfab7-0cf9-4916-adb2-5fdafb038070",
    user_id="e8950cdc-4d5a-4412-b3f1-f9f7357609b7",
    type_name="Coach",
    type_role=bot_type,
    instance_context=bot_instance,
    instance_starter=bot_starter
)

In [4]:
print(bot.start())

['<p>Hallo, wie schön, dass Du hier bist! 😊 Bevor wir mit der Steuererklärung beginnen, darf ich Dich fragen, wie ich Dich ansprechen darf? </p>']


URL to be handed out to the user: If you are following the instructions to deploy your chatbot(s) to pythonanywhere, this is the URL to access your chatbot.

##### Generic URL
https://[your pythonanywhere user name].pythonanywhere.com/[type id]/[user_id]/chat
##### For Example
https://monkey23.pythonanywhere.com/053e97a0-6a91-4589-8602-340aa47b6376/7515865e-4097-4dd7-9567-d3c7a4c1ed07/chat

#### Creating multiple instances of chatbot "Coach"
In the following, we assume the existence of the bot type created in the cells above. We show example code that will generate N bot instances of that type. Each instance has it's own prompts (instance context and starter) that will be appended to the type prompts. Most importantly, each instance has its own chat history.

In [5]:
import uuid
import time

In [6]:
# Amount of instances to be created
number_of_instances = 12

# Change the following to a list of hardcoded instance IDs if you want to use existing users.
user_ids = [str(uuid.uuid4()) for _ in range(number_of_instances)]

c  = 0 # counter for successful requests, don't change
error_c = 0 # counter for failed requests, don't change
for user_id in user_ids:
    bot = Chatbot(
        database_file="database/chatbot.db", 
        type_id="db4dfab7-0cf9-4916-adb2-5fdafb038070",
        user_id=user_id,
        instance_context=bot_instance,
        instance_starter=bot_starter
    )
    try:
        # each bot should have a first message to the user
        print(bot.start())
    except RuntimeError as error:
        print(error)
        error_c += 1
        continue
    c+=1
    time.sleep(15) #openai seems to produce more errors if we send the requests too fast.
    
print("successful: {}, failed: {}".format(c, error_c))


['<p>Hallo und herzlich willkommen! 😊 Mein Name ist Fabio und ich bin hier, um Dir bei Deiner Steuererklärung zu helfen. Wie darf ich Dich ansprechen?</p>']
['<p>Hallo und herzlich willkommen! Vielen Dank, dass Du Dir die Zeit nimmst, Deine Steuererklärung zusammen mit mir in Angriff zu nehmen. 😊 Bevor wir starten, darf ich fragen, wie ich Dich ansprechen darf?</p>']
['<p>Hallo und herzlich willkommen zu unserer Steuer-Coaching-Session! 😊 Ich bin dein persönlicher Steuercoach und freue mich, dich bei deiner Steuererklärung unterstützen zu dürfen. Wie darf ich dich ansprechen? </p>']
['<p>Herzlich willkommen! Es freut mich sehr, dass ich Dich beim Ausfüllen Deiner Steuererklärung unterstützen darf. Zunächst hätte ich eine kleine Frage: Wie darf ich Dich ansprechen? 😊</p>']
['<p>Hallo dort! 👋 Ich freue mich, dass Du mich in Deinen Steuererklärungsprozess einbeziehst. Wie kann ich Dich ansprechen? 😊</p>']
['<p>Ein herzliches Willkommen! 😊 Es freut mich sehr, dich bei deiner Steuererklärun

#### Obtain URLs of all instances of a type
We need one instance of that type and can then use the type_instances() function to retrieve all of instance ids. Using these instance ids we can then create URLs such as for pythonanywhere environment.

In [7]:
pythonanywhere_username = "a4e5bcd8"
type_id = "db4dfab7-0cf9-4916-adb2-5fdafb038070"
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id=type_id,
    user_id=user_ids[0]
)

for user_id in bot.type_instances():
    print("https://{}.pythonanywhere.com/{}/{}/chat".format(pythonanywhere_username, type_id, user_id))

https://a4e5bcd8.pythonanywhere.com/db4dfab7-0cf9-4916-adb2-5fdafb038070/173fa2f7-a39a-4202-aa5f-cdee2e00e7ae/chat
https://a4e5bcd8.pythonanywhere.com/db4dfab7-0cf9-4916-adb2-5fdafb038070/26d1ba99-d420-4ae8-bfce-489c5e26697b/chat
https://a4e5bcd8.pythonanywhere.com/db4dfab7-0cf9-4916-adb2-5fdafb038070/2efb115a-8e46-404b-a9ea-8f7177c37e21/chat
https://a4e5bcd8.pythonanywhere.com/db4dfab7-0cf9-4916-adb2-5fdafb038070/30ef45cb-9642-4d38-97a5-7aaeb24a6f2d/chat
https://a4e5bcd8.pythonanywhere.com/db4dfab7-0cf9-4916-adb2-5fdafb038070/31d5da72-9867-42da-8f0d-62503835abe6/chat
https://a4e5bcd8.pythonanywhere.com/db4dfab7-0cf9-4916-adb2-5fdafb038070/527ef3e5-9ada-4ec6-9295-0ecfb8409b5f/chat
https://a4e5bcd8.pythonanywhere.com/db4dfab7-0cf9-4916-adb2-5fdafb038070/60cae08e-e438-46eb-b178-068d34edf551/chat
https://a4e5bcd8.pythonanywhere.com/db4dfab7-0cf9-4916-adb2-5fdafb038070/7423377b-2707-46ea-b838-f1e182459ed7/chat
https://a4e5bcd8.pythonanywhere.com/db4dfab7-0cf9-4916-adb2-5fdafb038070/ccbd60a